In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.stats as stats
import numpy as np
import datetime as dt
from IPython.parallel import Client
import scipy.signal as signal

%matplotlib inline

In [2]:
ipclient = Client()
print (ipclient.ids)
#ipview = ipclient[:]
ipview = ipclient.load_balanced_view()

[0, 1, 2, 3]


### Initilize Database

In [3]:
client = MongoClient()
db = client.VentDyssynchrony_db
breath_data = db.BreathData_collection
patient_data = db.PatientData_collection
log_data = db.LogData_collection
vent_data = db.VentSettings_collection
RN_data = db.RNData_collection
RT_data = db.RTData_collection
Lab_data = db.LabData_collection

### Open RawData and Populate Data Base

In [4]:
path='E:\Research_Data\Vent_Dyssynchrony\Data\Raw_Data'
#path='Z:\Research_Data\Vent_Dyssynchrony\Data\Raw_Data'
#path = r'C:\Users\sottilep\Documents\Vent_Dyssynchrony\Raw_Data'

In [5]:
@ipview.parallel(block=True)
def data_entry(data_files):
    import pandas as pd
    import scipy.stats as stats
    import numpy as np
    import matplotlib.pyplot as plt
    import datetime as dt
    from pymongo import MongoClient
    import pymongo
    import sys
    import scipy.signal as signal
    
    # Open Mongos for Each Engine for Parallel Inputs
    client = MongoClient()
    db = client.VentDyssynchrony_db
    breath_data = db.BreathData_collection
    patient_data = db.PatientData_collection
    log_data = db.LogData_collection
    vent_data = db.VentSettings_collection
    RN_data = db.RNData_collection
    RT_data = db.RTData_collection
    Lab_data = db.LabData_collection
    
    # Define Location Function
    def get_location(group, errors):
        try:
            location = int(pd.to_datetime(group.DateTime.min(), coerce=True).value)
        except:
            location = 0
            errors.appends('location conversion failed')
        return (location, errors)
    
    # Define Elapse Time Calculator:
    def get_ElapseTime(df, errors):
        try:
            elapsetime = pd.to_timedelta(df.DateTime.max()-df.DateTime.min(), unit='s')
        except:
            elapsetime = 0
            errors.append('DateTime NAT')
            print (patients, files, df.DateTime.max(), df.DateTime.min())
        return (elapsetime, errors)
    
    # Define Pressure and Flow Extrema Calculator (returns tuple(value, time, location))
    def pres_flow_extrema(group):
        
        types = ['Flow (l/min)', 'Paw (cmH2O)']
        max_flow = []
        min_flow = []
        max_pressure = []
        min_pressure = []

        for t in types:
            leng = int(len(group[t]) / 15) + 1
            max_peaks = signal.argrelmax(group[t].values, order = leng)
            min_valleys = signal.argrelmin(group[t].values, order = leng)

            if (len(max_peaks[0]) < 1 or len(min_valleys[0]) < 1):
                max_peaks = signal.argrelmax(group[t].values, order = 1)
                min_valleys = signal.argrelmin(group[t].values, order = 1)
                
            for peaks in max_peaks[0]:
                if t == 'Flow (l/min)':
                    max_flow.append([group['Flow (l/min)'].iloc[peaks], str(group.ElapseTime.iloc[peaks]), peaks])
                else:
                    max_pressure.append([group['Paw (cmH2O)'].iloc[peaks], str(group.ElapseTime.iloc[peaks]), peaks])
            for valleys in min_valleys[0]:
                if t == 'Flow (l/min)':
                    min_flow.append([group['Flow (l/min)'].iloc[valleys], str(group.ElapseTime.iloc[valleys]), valleys])
                else:
                    min_pressure.append([group['Paw (cmH2O)'].iloc[valleys], str(group.ElapseTime.iloc[valleys]), valleys])

        return (max_pressure, max_flow, min_pressure, min_flow)

    # Define File Cleanup Function
    def file_cleaner(file_path, file_type):
        errors = []
        
        if file_type == 'wave' or file_type == 'breath':  
            try:
                df = pd.read_csv(file_path, sep = '\t', header = 1, error_bad_lines=False, parse_dates={'DateTime':['Date', 'HH:MM:SS']},
                                      infer_datetime_format=True, dayfirst=True, na_values = '--')
            except:
                errors.append('Error on CSV load')
                print(patients, files)
                
            try:
                df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True, infer_datetime_format=True, coerce=True)
            except:
                errors.append('Error in DateTime Load')
                print(patients, files, df.DateTime, sys.exc_info()[0])
            
            if file_type == 'wave':
                df['ElapseTime'] = pd.to_timedelta(df['Time(ms)'], unit='ms')
                df.reset_index(inplace=True, drop=False)

                if stats.linregress(df.index, df['Time(ms)'])[2] < 0.95:
                    errors.append('time frame not continous, ' + 'r = ' + str(stats.linregress(df.index, df['Time(ms)'])[2]))
                    df.plot(x='index', y='Time(ms)')
                    plt.title(file_path)

                breath_diff = np.diff(df.Breath)
                if breath_diff.any() != 1 or 0: 
                    errors.append('breath count not continous')
                    df.plot(x='index', y='Breath')
                    plt.title(file_path)

                df.drop(['Paux (cmH2O)', 'CO2 (mmHg)', 'Comment', 'Time(ms)'], axis=1, inplace=True)
            
            elif file_type == 'breath':
                df.drop(['Flow Pattern', 'MMV (l/min)', 'ExpMinVol Lo (l/min)', 'ExpMinVol Hi (l/min)', '%MinVol (%)','Pressure Hi (cmH2O)', 
                                 'Pcontrol (cmH2O)', 'Paux/Paw', 'Int. Controller','f cmv (b/min)', 'f simv (b/min)', 'Rate Hi (b/min)', 
                                 'Oxygen Lo (%)', 'Oxygen Hi (%)', 'P0.1 (cmH2O)', 'Pinsp (cmH2O)', 'PetCO2 (mmHg)', 'SpO2 (%)', 
                                 'Pulse (l/min)', 'HLI (%)', 'Variab. Index (%)', 'Oxygen (%).1', 'Rcexp (s)', 'Rcinsp (s)', 'WOB (J/l)', 
                                 'PTP (cmH2O*s)', '!High Pressure', '!Disconn. Patient', '!Apnea', '!Low MinVol', '!High MinVol', 
                                 '!High Rate', '!General Alarm', '!Silence', '!Fail to Cycle', '!Disconn. Vent.', '!Loss of PEEP', 
                                 '!Oxygen conc.', '!Operator', '!Gas Supply', '!SpezAlarm', 'ETS (%)', 'Body Wt (kg)','I:E.1'], axis=1, inplace=True)
                df.rename(columns={'Vt (ml)':'Set Vt', 'Peep (cmH2O)':'PEEP', 'Mode':'Vent Mode', 'f cmv (b/min)':'Set RR', 'Oxygen (%)':'FiO2'}, inplace=True)
                df.reset_index(inplace=True, drop=False)
            
        else:
            try:
                if file_type == 'lab': 
                    df = pd.read_csv(path +'\\'+ patients +'\\'+ files, na_values='--', infer_datetime_format=True,
                                     parse_dates={'DateTime':['Date/Times']}, error_bad_lines=False)
                else: df = pd.read_csv(path +'\\'+ patients +'\\'+ files, na_values='--', infer_datetime_format=True,
                                 parse_dates=['DateTime'], error_bad_lines=False)
            except:
                errors.append('Error on CSV load')
                #df = pd.DataFame()
                print (patients, files, sys.exc_info()[:1])

            try:
                df['DateTime'] = pd.to_datetime(df['DateTime'], infer_datetime_format=True, coerce=True)
            except:
                errors.append('Error in DateTime Load')
                print(patients, files, df.DateTime, sys.exc_info()[:1]) 
            
            df.dropna(axis = 0, how = 'all', inplace=True)
            df.reset_index(inplace=True, drop=False)
            
            if file_type == 'rn':
                try:
                    df.drop(['Unnamed: 35', 'A-line MAP.1'], axis=1, inplace = True)
                except:
                    pass
                try:
                    df.rename(columns={'Set Vt (mLs)':'Set Vt', 'HOB Position':'HOB', 'PEEP (cmH20)':'PEEP', 'FIO2':'FiO2'}, inplace=True)
                except:
                    pass
                try:
                    df['PEEP'] = df['PEEP'].str.rstrip(' cmH2O')
                except:
                    pass
                df.replace(to_replace={'Vent Mode':{'APV;CMV':'APVCMV', 'PRVC':'APVCMV', 'PSV;CPAP':'SPONT', 'CPAP':'SPONT',
                                                'CMV':'APVCMV', 'PCMV':'PCV', 'APV':'APVCMV', 'CMV;APV':'APVCMV'}}, inplace=True)

            elif file_type == 'rt':
                try:
                    df.drop(['Auto Flow', 'Exhaled Vt', 'Cuff Pressure','Actual Ve','Set Ve', 'Unnamed: 25', 'Spontaneous Vt'], axis=1, inplace = True)
                except:
                    pass
                try:
                    df['PEEP'] = df['PEEP'].str.rstrip(' cmH2O')
                except:
                    pass
                df.replace(to_replace={'Vent Mode':{'APV;CMV':'APVCMV', 'PRVC':'APVCMV', 'PSV;CPAP':'SPONT', 'CPAP':'SPONT',
                                                'CMV':'APVCMV', 'PCMV':'PCV', 'APV':'APVCMV', 'CMV;APV':'APVCMV'}}, inplace=True)

            elif file_type == 'lab':
                for head in df.columns:
                    new_head = head.lstrip()
                    new_head = head.lower()
                        
                    if '.' in head:
                        df.rename(columns={head:new_head.split('.')[0]+new_head.split('.')[1]}, inplace=True)
                    else:
                        df.rename(columns={head:new_head}, inplace=True)
                        
                    if head == 'DateTime' or new_head == 'datetime':
                        df.rename(columns={'datetime':'DateTime'}, inplace=True)
        
        try:
            if df.DateTime.any() == np.nan: 
                errors.append('DateTime with NAN')
        except:
            print (patients, files, sys.exc_info()[:1])
                  
        return (df, errors)

    # Main Loop through each file 
    for data in data_files:  
        path = data[0]
        patients = data[1]
        files = data[2]
       
        if 'Waveform' in files:
            df, errors = file_cleaner(path + '\\' + patients + '\\' + files, 'wave')
            groups = df.groupby('Breath')
            for name, group in groups:
                
                try:
                    location = int(group.DateTime.astype(np.int64).min())
                except:
                    location = 0
                    errors.append('location conversion failed')
                
                max_pressure, max_flow, min_pressure, min_flow = pres_flow_extrema(group)
                
                try: 
                    insp_time = group[group.Status != 0]['ElapseTime'].dt.to_pytimedelta().max().total_seconds()-group[group.Status != 0]['ElapseTime'].dt.to_pytimedelta().min().total_seconds()
                except:
                    insp_time = 0
                    errors.append('could not calculate iTime: ' + str(name))
                
                try: 
                    breath_time = group.ElapseTime.dt.to_pytimedelta().max().total_seconds()-group.ElapseTime.dt.to_pytimedelta().min().total_seconds()
                except:
                    breath_time = 0
                    errors.append('could not calculate breath_time: ' + str(name))
        
                data_document = {
                            '_id': patients + '\\WF' + str(group.Breath.min()) +'\\' + str(group.DateTime.min()),
                            'patientID': patients,
                            'file_name': patients + '\\' + files,
                            'breath_number':int(group.Breath.max()),
                            'start_time': group.DateTime.dt.to_pydatetime().min(),  #str(group.DateTime.min())
                            'end_time': group.DateTime.dt.to_pydatetime().max(),
                            'location': [location, 0],
                            'characteristics':{'breath_time':breath_time,
                                               'insp_time': insp_time,
                                               'exp_time': breath_time - insp_time,
                                               'peak_pressure': group['Paw (cmH2O)'].max(),
                                               'min_pressure':group['Paw (cmH2O)'].min(),
                                               'peak_insp_flow': group[group.Status == 0]['Flow (l/min)'].max(),
                                               'min_exp_flow': group[group.Status == 1]['Flow (l/min)'].max(),
                                               'max_vol': group['Volume (ml)'].max(),
                                               'min_vol': group['Volume (ml)'].min(),
                                               'end_insp_vol': group[group.Status == 0]['Volume (ml)'].tail(10).min(),
                                               'max_pressure': str(max_pressure),
                                               'max_flow': str(max_flow),
                                               'min_pressure': str(min_pressure), 
                                               'min_flow': str(min_flow) }, 
                            'data_frame': {'DateTime': group.DateTime.dt.to_pydatetime().tolist(),
                                           'Time': group.ElapseTime.values.tolist(),
                                           'Status': group.Status.values.tolist(),
                                           'Paw': group['Paw (cmH2O)'].values.tolist(),
                                           'Flow': group['Flow (l/min)'].values.tolist(),
                                           'Volume': group['Volume (ml)'].values.tolist()}}  
                
                try:
                    breath_data.insert(data_document)    
                except:
                    print(patients, files, sys.exc_info()[:1])
            elapsetime = df.ElapseTime.max()
        
        elif 'Breath' in files:
            df, errors = file_cleaner(path + '\\' + patients + '\\' + files, 'breath')
            groups = df.groupby('index')
            for name, group in groups:    
                location, errors = get_location(group, errors)
                group.dropna(axis=1, how='any', inplace=True)
                if group.size > 0:
                    try:
                        data_document = {
                            '_id': patients + '\\Vent' + str(group.index.min()) +'\\' + str(group.DateTime.min()),
                            'patientID': patients,
                            'file_name': patients + '\\' + files,
                            'breath_number':int(group.index.max()),
                            'breath_time': group.DateTime.dt.to_pydatetime().min(),
                            'vent_settings': group.to_dict(orient='records'),
                            'location': [location, 0]}
                  
                        try:
                            vent_data.insert(data_document)
                        except pymongo.errors.DuplicateKeyError:
                            pass
                    except:
                        errors.append('No DateTime on Insert '+patients+'\\'+files+str(name))
            elapsetime, errors = get_ElapseTime(df, errors)
                
        elif 'RN' in files:
            df, errors = file_cleaner(path + '\\' + patients + '\\' + files, 'rn')
            groups = df.groupby('index')
            for name, group in groups:
                location, errors = get_location(group, errors)
                group.dropna(axis=1, how='any', inplace=True)
                if group.size > 0:
                    try:
                        data_document = {
                            '_id': patients + '\\RN' + str(group.index.min()) + '\\' + str(group.DateTime.min()),
                            'patientID': patients,
                            'file_name': patients + '\\' + files,
                            'entry_number':int(group.index.max()),
                            'entry_time': group.DateTime.dt.to_pydatetime().min(),
                            'RN_entry': group.to_dict(orient='records'),
                            'location': [location, 0]}
                    
                        try:
                            RN_data.insert(data_document)
                        except pymongo.errors.DuplicateKeyError:
                            pass
                    
                    except:
                        errors.append('Not DateTime' +patients+files)
                    
            elapsetime, errors = get_ElapseTime(df, errors)
        
        elif 'RT' in files:
            df, errors = file_cleaner(path + '\\' + patients + '\\' + files, 'rt')
            groups = df.groupby('index')
            for name, group in groups: 
                location, errors = get_location(group, errors)
                group.dropna(axis=1, how='any', inplace=True)
                if group.size > 0:
                    data_document = {
                            '_id': patients + '\\RT' + str(group.index.min()) +'\\' + str(group.DateTime.min()),
                            'patientID': patients,
                            'file_name': patients + '\\' + files,
                            'entry_number':int(group.index.max()),
                            'entry_time': group.DateTime.dt.to_pydatetime().min(),
                            'RT_entry': group.to_dict(orient='records'),
                            'location': [location, 0]}
                    try:
                        RT_data.insert(data_document)
                    except pymongo.errors.DuplicateKeyError:
                        pass
            elapsetime, errors = get_ElapseTime(df, errors)
        
        elif 'Lab' in files:
            df, errors = file_cleaner(path + '\\' + patients + '\\' + files, 'lab') 
            groups = df.groupby('index')
            for name, group in groups: 
                location, errors = get_location(group, errors)
                group.dropna(axis=1, how='any', inplace=True)
                if group.size > 0:
                    data_document = {
                            '_id': patients + '\\Lab' + str(group.index.min()) +'\\' + str(group.DateTime.min()),
                            'patientID': patients,
                            'file_name': patients + '\\' + files,
                            'entry_number':int(group.index.max()),
                            'entry_time': group.DateTime.dt.to_pydatetime().min(),
                            'Lab_entry': group.to_dict(orient='records'),
                            'location': [location, 0]}
                    try:
                        Lab_data.insert(data_document)
                    except pymongo.errors.DuplicateKeyError:
                        pass
            elapsetime, errors = get_ElapseTime(df, errors)

        
        log_document = {
                    '_id': patients + '\\' + files,
                    'patientID': patients,
                    'file_name': files,
                    'update_date': dt.datetime.now(),
                    'n_data': len(df.index),
                    'start_time':df.DateTime.dt.to_pydatetime().min(),
                    'end_time':df.DateTime.dt.to_pydatetime().max(),
                    'elapse_time': elapsetime.total_seconds(),
                    'file_errors': errors,
                    'location': [pd.to_datetime(df.DateTime.min(), coerce=True).value, 0]}
        log_data.insert(log_document)
        
    return

In [8]:
directory = os.listdir(path)
data_files = []
all_files = []

for patients in directory:
    file_list = os.listdir(path + '\\' + patients)

    for files in file_list:
        all_files.append([path, patients, files])
        if '.txt' in files and os.path.getsize(path +'\\'+ patients +'\\'+ files) > 1024:
            if log_data.find({'_id': patients + '\\' + files}).count() == 0:
                if 'Waveform' in files or 'Breath' in files or 'edit' in files: 
                    data_files.append([path, patients, files])               

df = data_entry(data_files)

In [9]:
data_files

[['E:\\Research_Data\\Vent_Dyssynchrony\\Data\\Raw_Data',
  'P122',
  'editedRN Data.txt'],
 ['E:\\Research_Data\\Vent_Dyssynchrony\\Data\\Raw_Data',
  'P122',
  'editedRT Data.txt'],
 ['E:\\Research_Data\\Vent_Dyssynchrony\\Data\\Raw_Data',
  'P122',
  'editLab Data.txt']]

##### Ensure Indexes

In [10]:
for sheets in db.collection_names(include_system_collections=False):
    print(sheets, pymongo.collection.Collection(db, sheets).count())
    pymongo.collection.Collection(db, sheets).ensure_index([('location', pymongo.GEO2D)],  min=0, max=1577836800000000000, name='Location')
    pymongo.collection.Collection(db, sheets).ensure_index([('patientID', pymongo.TEXT)],  name='patientID')


breath_data.ensure_index([('start_time', pymongo.ASCENDING)], name='StartTime')
breath_data.ensure_index([('breath_number', pymongo.ASCENDING)], name='breath_number')
breath_data.ensure_index([('characteristics.breath_time', pymongo.ASCENDING)], name='breath_time')
breath_data.ensure_index([('characteristics.exp_time', pymongo.ASCENDING)], name='exp_time')
breath_data.ensure_index([('characteristics.insp_time', pymongo.ASCENDING)], name='insp_time')
breath_data.ensure_index([('vent_settings.vent_mode', pymongo.ASCENDING)], name='vent_mode')
breath_data.ensure_index([('vent_settings.set_VT', pymongo.ASCENDING)], name='set_VT')
breath_data.ensure_index([('characteristics.peak_pressure', pymongo.ASCENDING)], name='peak_pres')
breath_data.ensure_index([('characteristics.max_vol', pymongo.ASCENDING)], name='max_vol')
breath_data.ensure_index([('vent_settings.distance', pymongo.ASCENDING)], name='distance')

vent_data.ensure_index([('vent_settings.Vent Mode', pymongo.ASCENDING)], name='Vent Mode')
vent_data.ensure_index([('vent_settings.f total (b/min)', pymongo.ASCENDING)], name='f')
vent_data.ensure_index([('vent_settings.PEEP', pymongo.ASCENDING)], name='PEEP')
vent_data.ensure_index([('vent_settings.FiO2', pymongo.ASCENDING)], name='FiO2')

BreathData_collection 1299170
LabData_collection 2076
LogData_collection 518
PatientData_collection 21
RNData_collection 8743
RTData_collection 7946
TestData_collection 20117
ValidatedData_collection 2388
VentSettings_collection 1370409


'FiO2'

### Link Breath Data to Vent Settings

In [11]:
@ipview.parallel(block=True)
def link_data(breaths):
    from pymongo import MongoClient
    import pymongo
    import sys

    # Open Mongos for Each Engine for Parallel Inputs
    client = MongoClient()
    db = client.VentDyssynchrony_db
    breath_data = db.BreathData_collection
    vent_data = db.VentSettings_collection
    
    for breath in breaths:
        data = vent_data.find_one({'patientID': breath['patientID'],
                               'vent_settings.Vent Mode': {'$exists':1},
                               'vent_settings.f total (b/min)': {'$exists':1},
                               'vent_settings.PEEP': {'$exists':1},
                               'vent_settings.FiO2': {'$exists':1},
                               'location': {'$near': breath['location']}
                              })
        
  
        if len(data)!=0:
            #data.rewind()
            #data = data[0]
            distance = breath['location'][0]-data['location'][0]
            
            data_types = {
                        'reference_doc': "data['_id']",
                        'file_name': "data['file_name']",
                        'distance': "distance/1000000000",
                        'vent_mode':"data['vent_settings'][0]['Vent Mode']",
                        'set_rate': "data['vent_settings'][0]['f total (b/min)']",
                        'PEEP':"data['vent_settings'][0]['PEEP']",
                        'FiO2': "data['vent_settings'][0]['FiO2']",
                        'set_VT': "data['vent_settings'][0]['Set Vt']",
                        'p_peak': "data['vent_settings'][0]['P peak (cmH2O)']",
                        'p_plat': "data['vent_settings'][0]['P plateau (cmH2O)']",
                        'p_mean': "data['vent_settings'][0]['P mean (cmH2O)']",
                        'leak': "data['vent_settings'][0]['Vt leak (ml)']",
                        'trigger': "data['vent_settings'][0]['Trigger']",
                        'ramp': "data['vent_settings'][0]['Ramp (ms)']",
                        'ti': "data['vent_settings'][0]['TI (s)']",
                        'te': "data['vent_settings'][0]['TE (s)']",
                        'ie': "data['vent_settings'][0]['I:E']",             
                        'compliance': "data['vent_settings'][0]['Cstat (ml/cmH2O)']"}

            document = {'load_errors':[]}       

            for items in data_types:
                try:
                    document.update({items:eval(data_types[items])})
                except KeyError:
                    document['load_errors'].append(items)

            breath_data.update({'_id':breath['_id']},
                              {'$set':{ 'vent_settings': document
                                }})
        else:
            breath_data.update({'_id':breath['_id']},
                              {'$set':{ 'vent_settings.load_errors': 'on matching breath.txt', 'vent_settings.distance':-6000
                                }})

    return

In [12]:
breaths = breath_data.find({'vent_settings.distance':{'$exists':0}},
                           {'_id':1, 'patientID':1, 'file_name':1, 'start_time':1, 'end_time':1, 'breath_number':1, 'location':1}).limit(200000)
link_data(breaths)

[]

In [13]:
breath_data.find({'vent_settings.distance':{'$exists':0}}).count()

0

## Add Demographic Data

In [30]:
demo_df = pd.DataFrame.from_csv('E:\Research_Data\Vent_Dyssynchrony\Data\Demographic Data v2.csv')
db.drop_collection('PatientData_collection')
date_col = ['Admission Date/Time', 'ICU Admit Date/Time', 'ICU Discharge Date/Time', 'Hospital Discharge Date/Time', 
             'Intubation Date/Time', 'Started Recording', 'End MV Date/Time']

for col in date_col:
    demo_df[col] = pd.to_datetime(demo_df[col])
    
demo_df.reset_index(inplace=True)
demo_df.rename(columns={'Age':'age', 'Gender':'gender', 'Race':'race','Admission Dx':'admit_dx', 
                        'Admission Date/Time':'admit_date', 'ICU Admit Date/Time':'icu_admit_date', 
                        'ICU Discharge Date/Time':'icu_dc_time', 'Hospital Discharge Date/Time':'hospital_dc_time', 
                        'Intubation Date/Time':'intubation_date', 'Started Recording':'recording_date', 'End MV Date/Time':'extubation_date',
                        'Vent Days':'vent_days', 'Vent Free Days':'vent_free_days', 'ICU LOS':'icu_los', 'Hospital LOS':'hosp_los',
                        'APACHE II Score':'apache_score', 'Initial PaO2':'init_paO2', 'Initial FIO2':'init_fio2', 'P/F Ratio':'p_to_f',
                        'CXR Appearance':'cxr', 'Discharge Location':'dc_location', 'Height(cm)':'height', 'Weight(kg)':'weight',
                        'Hours on Vent':'hour_on_vent', 'Hours Recorded':'hours_recorded', 'Reintubated':'reintubated', 'Tracheostomy':'trach',
                        'NMB':'nmb', 'Proned':'proned', 'iNO':'ino', 'Vasopressors':'vasopressors', 'PBW':'pbw',
                        'Percent Recorded': 'percent_recorded', 'Percent Recorded Once Entrolled':'percent_recorded_enrolled',
                        'Time to Enrollement':'time_to_enroll', 'Study ID':'_id'}, inplace=True)

demo_df.drop(['AUDIT-C Score'], axis=1, inplace=True)
demo_df['_id'] = demo_df['_id'].apply(lambda x: 'P' +str(x))

In [31]:
patient_data.insert(demo_df.to_dict(orient='records'))

['P100',
 'P101',
 'P102',
 'P103',
 'P104',
 'P105',
 'P106',
 'P107',
 'P108',
 'P109',
 'P110',
 'P112',
 'P113',
 'P114',
 'P115',
 'P116',
 'P117',
 'P118',
 'P119',
 'P120',
 'P121',
 'P122']

## Database Clean-Up and Data Verification

##### Remove Empty RT and RN Data

In [16]:
data_sets = [RN_data, RT_data]

for data in data_sets:
    if data.name == 'RNData_collection': 
        doc = 'RN_entry'
    else: 
        doc = 'RT_entry'

    results = data.find({}, {'_id':1, doc:1})
    unique_keys = set()
    
    for items in results:
        if len(items[doc][0].keys()) < 3: 
           data.remove(items['_id'])

##### Ensure Vent_Mode Linked to Close Data Point (<10min) from Breath Data, RT Data, RN Data

In [17]:
print(list(breath_data.aggregate([{'$match': {'$or':[{'vent_settings.distance': {'$gte': 600}}, {'vent_settings.distance': {'$lte':-600}}]}},
                        {'$group': {'_id': '$patientID','count': {'$sum':1}, 'avg':{'$avg':'$vent_settings.distance'}}}])))

[{'avg': 3209.9607423269094, '_id': 'P117', 'count': 1401}, {'avg': 2315.7755, '_id': 'P107', 'count': 2000}, {'avg': 6.8201058201058204, '_id': 'P119', 'count': 567}, {'avg': -1884.0, '_id': 'P116', 'count': 1}, {'avg': -6000.0, '_id': 'P104', 'count': 5095}, {'avg': -6818.807822172201, '_id': 'P121', 'count': 3554}, {'avg': 1398.7679045092839, '_id': 'P102', 'count': 754}, {'avg': -8035.02662627551, '_id': 'P114', 'count': 6272}]


In [18]:
@ipview.parallel(block=True)
def find_closer(results):
    from pymongo import MongoClient
    import pymongo

    # Open Mongos for Each Engine for Parallel Inputs
    client = MongoClient()
    db = client.VentDyssynchrony_db
    breath_data = db.BreathData_collection
    RN_data = db.RNData_collection
    RT_data = db.RTData_collection
    
    data_sets = [RN_data, RT_data]
    near = []
    for items in results:
        
        for data in data_sets:
            if data.name == 'RNData_collection': 
                doc = 'RN_entry'
            else: 
                doc = 'RT_entry'

            RNRT_data = data.find({'patientID':items['patientID'], doc+'.Vent Mode':{'$exists':1}, 'location':{'$near': items['location']}}).limit(1)
            near.append(RNRT_data.count())
            if RNRT_data.count() != 0:
                
                if (RNRT_data[0]['location'][0] - items['location'][0])/1000000000 < items['vent_settings']['distance']:
                    old_dist = items['vent_settings']['distance']
                    breath_data.update({'_id':items['_id']},
                              {'$set':{'vent_settings.vent_mode': RNRT_data[0][doc]['Vent Mode'], 
                                       'vent_settings.distance': (RNRT_data[0]['location'][0] - items['location'][0])/1000000000,
                                       'vent_settings.old_distance': old_dist
                                }})
    return

In [19]:
results = breath_data.find({'$and':[{'vent_settings.old_distance':{'$exists':0}},
                                    {'$or':[{'vent_settings.distance': {'$gte': 600}}, {'vent_settings.distance': {'$lte':-600}}]}]},
                          {'_id':1, 'patientID':1, 'location':1, 'vent_settings.distance':1, 'vent_settings.vent_mode':1})

#find_closer(results)

In [20]:
breath_data.aggregate([{'$match': {'vent_settings.distance': {'$exists': 1}}},
                        {'$group': {
                            '_id':'$patientID', 
                            'avg':{'$avg':'$vent_settings.distance'},
                            'max':{'$max':'$vent_settings.distance'},
                            'min':{'$min':'$vent_settings.distance'}}}])

In [21]:
breath_data.aggregate([{'$match': {'vent_settings.old_distance': {'$exists': 1}}},
                        {'$group': {'_id': '$patientID','count': {'$sum':1}}}])

##### Check for Long Breaths (>10s)

In [22]:
breath_data.find({'characteristics.breath_time':{'$gte':10}}).count()

401

##### Check for Small Breaths (<50cc - likely ineffective trigger)

In [23]:
breath_data.find({'characteristics.max_vol':{'$lte':100}}).count()

24383

##### Check for Large Breaths (>2000cc)

In [24]:
breath_data.find({'characteristics.max_vol':{'$gte':2000}}).count()

113

### Additional Breath Analysis

In [ ]:
breath_data.find({'vent_settings.breath_type':'APVCMV'}).count()

##### Find Double Stacked Breaths

In [25]:
breath_data.find({'characteristics.breath_time':{'$lte':1.5}}).count()

75548

In [26]:
breath_data.find({'characteristics.exp_time':{'$exists':0}}).count()

0

In [27]:
@ipview.parallel(block=True)
def find_exp_time(results):
    from pymongo import MongoClient
    import pymongo

    # Open Mongos for Each Engine for Parallel Inputs
    client = MongoClient()
    db = client.VentDyssynchrony_db
    breath_data = db.BreathData_collection

    for items in results:
        exp_time = items['characteristics']['breath_time'] - items['characteristics']['insp_time']
        breath_data.update({'_id':items['_id']},
                           {'$set': {'characteristics.exp_time': exp_time}})

In [28]:
results = breath_data.find({'characteristics.exp_time':{'$exists':0}}).limit(100000)
find_exp_time(results)

[]

In [29]:
breath_data.find({'characteristics.exp_time':{'$lte':0.5}}).count()

44013

##### Find Flow Limited Breaths

##### Find Pressure Limited Breaths

##### Find Ineffective Triggers

In [ ]:
breath_data.find({'characteristics.max_vol':{'$lte':100}}).count()

##### Find Breaths with NMB

##### Link Breaths with Recent RASS (within 30min)